# Import modules

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## MINIST Test (Using FastMINIST)

MixedNational Institute of Standards and Technology Database

In [2]:
# Check if cuda is enable
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
# Download training data and test data
# url = ""
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
# url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz"
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
# Model definitoin

class NetworkModel(nn.Module):
    def __init__(self):
        super(NetworkModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NetworkModel().to(device)
print(model)

NetworkModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
# loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
# dataloaders

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for (X, y) in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [21]:
# training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # predition error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # save every 100
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [19]:
# test function
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy： {(100 * correct):>0.1f}%, Average loss: {test_loss:>8f} \n")

In [22]:
epochs = 5
for i in range(epochs):
    print(f"---- Epoch {i + 1} ----")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done")

---- Epoch 1 ----
loss:     nan [    0/60000]
loss:     nan [ 6400/60000]
loss:     nan [12800/60000]
loss:     nan [19200/60000]
loss:     nan [25600/60000]
loss:     nan [32000/60000]
loss:     nan [38400/60000]
loss:     nan [44800/60000]
loss:     nan [51200/60000]
loss:     nan [57600/60000]
Test Error: 
 Accuracy： 10.0%, Average loss:      nan 

---- Epoch 2 ----
loss:     nan [    0/60000]
loss:     nan [ 6400/60000]
loss:     nan [12800/60000]
loss:     nan [19200/60000]
loss:     nan [25600/60000]
loss:     nan [32000/60000]
loss:     nan [38400/60000]
loss:     nan [44800/60000]
loss:     nan [51200/60000]
loss:     nan [57600/60000]
Test Error: 
 Accuracy： 10.0%, Average loss:      nan 

---- Epoch 3 ----
loss:     nan [    0/60000]
loss:     nan [ 6400/60000]
loss:     nan [12800/60000]
loss:     nan [19200/60000]
loss:     nan [25600/60000]
loss:     nan [32000/60000]
loss:     nan [38400/60000]
loss:     nan [44800/60000]
loss:     nan [51200/60000]
loss:     nan [57600/6